# Job creation and submission to Azure

**!Important!** 
This script uses **Azure SDKv2** for the creation of the training job. 
The notebook used for uploading the data uses **Azure SDKv1**

In [ ]:
import os
from azure.ai.ml import MLClient, Input
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment
from azure.ai.ml import command

from src.utils import load_json, load_yaml

In [ ]:
azure_connect_conf = load_json("./../../configs/azure_connect_example.json")
azure_connect_conf

In [ ]:
azure_job_conf = load_yaml("./../../configs/azure_job.yaml")
dataset_conf = azure_job_conf["dataset"]
environment_conf = azure_job_conf["environment"]
job_conf = azure_job_conf["job"]
job_args_conf = job_conf["args"]

In [ ]:
# Authenticate and create a MLClient instance
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=azure_connect_conf["subscription_id"],
    resource_group_name=azure_connect_conf["resource_group"],
    workspace_name=azure_connect_conf["workspace_name"]
)

In [ ]:
# Check connection to workspace
ws = ml_client.workspaces.get(ml_client.workspace_name)
print(ws.name, ws.location, ws.resource_group, sep='\t')

In [ ]:
# Get created Dataset as Data Asset
dataset = ml_client.data.get(name=dataset_conf["name"], version=dataset_conf["version"])

print(dataset.name)
print(dataset.path)

In [ ]:
# Prepare Environment
env = Environment(
    name=environment_conf["name"],
    description=environment_conf["description"],
    image=environment_conf["image"],
    conda_file=environment_conf["conda_file"]
)

print(env)

In [ ]:
# Define jobs
job = command(
    code=job_conf["code"],  # Project folder (contains main.py)
    command=f"python main.py "
            f"--data_path ${{{{inputs.input_data}}}} "
            f"--wandb {job_args_conf['wandb']} "
            f"--wandb_proj_name {job_args_conf['wandb_proj_name']} "
            f"--wandb_run_name {job_args_conf['wandb_run_name']} "
            f"--train_images_path {job_args_conf['train_images_path']} "
            f"--train_masks_path {job_args_conf['train_masks_path']} "
            f"--test_images_path {job_args_conf['test_images_path']} "
            f"--test_masks_path {job_args_conf['test_masks_path']} "
            f"--config {job_args_conf['config']}",
    inputs={
        "input_data": Input(
            type="uri_folder",
            path=dataset.path,
            mode="ro_mount"
            )
    },
    environment=env,
    experiment_name=job_conf["experiment_name"],
    compute=job_conf["compute"],  # compute cluster name
    display_name=job_conf["azure_display_name"]  # run display name
    )
        
# Submit the job
ml_client.jobs.create_or_update(job)